In [39]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

LIBRARIES 

In [40]:
from sklearn.linear_model import LinearRegression,SGDRegressor,Ridge,Lasso,LogisticRegression
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import ConfusionMatrixDisplay,classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np


DOWNLOADING DATA

In [41]:
titanic=pd.read_csv("../input/titanic/train.csv")
print(titanic.info())


DATA INFO

In [42]:
X=titanic.drop("Survived",axis=1)
y=titanic.Survived
print(X.shape)
print(y.shape)
print(y.unique())

CHECKING FOR NULL VALUES

In [43]:
print(X.isnull().sum())
print(X.head())



REPLACING/FIXING NULL VALUES

In [44]:

impute=SimpleImputer(missing_values=np.nan,strategy="mean")
impute_age=impute.fit_transform(X["Age"].values.reshape(-1,1))

pd.DataFrame(impute_age).isnull().sum()

In [45]:
label=LabelEncoder()
gen_encoder=label.fit_transform(X["Sex"].values.reshape(-1,1))
embarked_encoder=label.fit_transform(X["Embarked"].values.reshape(-1,1))
cabin_encoder=label.fit_transform(X["Cabin"].values.reshape(-1,1))
pd.DataFrame(gen_encoder).isnull().sum()
pd.DataFrame(embarked_encoder).isnull().sum()

In [46]:
X=X.drop("Age",axis=1)
X["Age"]=impute_age

In [47]:
X=X.drop("Sex",axis=1)
X["Sex"]=gen_encoder

In [48]:
X=X.drop("Cabin",axis=1)
X["Cabin"]=cabin_encoder

In [49]:
X=X.drop("Embarked",axis=1)
X["Emabrked"]=embarked_encoder

In [50]:
print(X.head())
print(X.isnull().sum())

In [51]:

X.drop(["Name","Ticket","Fare"],axis=1,inplace=True)
X.plot.kde()

In [52]:
ss=StandardScaler()
X_ss=ss.fit_transform(X)

In [53]:
pd.DataFrame(X_ss).plot.kde()

SPLITTING DATASET

In [54]:
x_train,x_test,y_train,y_test=train_test_split(X_ss,y,test_size=0.2)

MODEl

In [55]:
lr=LogisticRegression()
lr_=lr.fit(x_train,y_train)
lr_.score(x_test,y_test)


In [56]:
y_pred_test=lr.predict(x_test)

In [57]:
classification_report(y_test,y_pred_test)

In [58]:
?LogisticRegression

In [59]:
rs=RandomizedSearchCV(lr,{"C":[1.0,10,0.01,0.12],"max_iter":[100,500,250],"warm_start":[True,False]})
rs_train=rs.fit(x_train,y_train)
rs_test=rs.fit(x_test,y_test)

In [60]:
rs_train.best_params_

In [61]:
rs_test.score(x_test,y_test)

In [62]:
rs_test.best_score_

**CONFUSION MATRIX**

In [63]:
cfmd=ConfusionMatrixDisplay.from_predictions(y_test,y_pred_test,values_format=".5g")

In [64]:
test_data=pd.read_csv("../input/titanic/test.csv")
impute_age_test=impute.fit_transform(test_data["Age"].values.reshape(-1,1))
test_data=test_data.drop("Age",axis=1)
test_data["Age"]=impute_age_test

In [65]:
gen_test_encode=label.fit_transform(test_data["Sex"].values.reshape(-1,1))
cabin_test_encode=label.fit_transform(test_data["Cabin"].values.reshape(-1,1))
embark_test_encode=label.fit_transform(test_data["Embarked"].values.reshape(-1,1))


In [67]:
test_data=test_data.drop("Sex",axis=1)
test_data["Sex"]=gen_test_encode

test_data=test_data.drop("Cabin",axis=1)
test_data["Cabin"]=cabin_test_encode

test_data=test_data.drop("Embarked",axis=1)
test_data["Embarked"]=embark_test_encode

test_data.drop(["Name","Ticket","Fare"],inplace=True,axis=1)
test_data_ss=ss.fit_transform(test_data)
y_pred_ss=lr.predict(test_data_ss)

In [68]:
test_data.head()